In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Spark_movies_links_task") \
    .config("spark.master", "spark://ZHUYANZHAO-HWPC.localdomain:7077") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.22") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.num", "1") \
    .getOrCreate()

In [ ]:
df_links = spark.read.option("header","true").csv("./links.csv")
df_links = df_links.withColumnRenamed("movieId", "movie_id") \
.withColumnRenamed("imdbId", "imdb_id") \
.withColumnRenamed("tmdbId", "tmdb_id")
df_links = df_links.na.drop(subset=['tmdb_id', 'imdb_id'])
df_links.count()
df_links.printSchema()
df_links.show(5, truncate=False)

In [ ]:
df_links.select(*df_links.columns).write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver", "com.mysql.cj.jdbc.Driver").option("dbtable", "movie_links") \
    .option("user", "root").option("password", "zhuyz928").save()